# Qualitative Prediction Visualization

This notebook provides qualitative visualization of baseline model predictions.

**Visualizations:**
1. Trajectory predictions (predicted vs ground truth)
2. BEV segmentation masks
3. Motion predictions for surrounding agents
4. Multi-modal latent representations (t-SNE/UMAP)
5. Attention maps (for V-JEPA/I-JEPA)

**Usage:**
- Run cells sequentially
- Visualizations saved to `results/baselines/visualizations/`

## Setup

In [ ]:
import sys
from pathlib import Path
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Circle, Rectangle, FancyArrow
from matplotlib.collections import PatchCollection

# Plotting style
sns.set_style("white")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

# Add project root to path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Import baseline models
from src.models.baselines import (
    CameraOnlyBaseline,
    LiDAROnlyBaseline,
    RadarOnlyBaseline,
    IJEPABaseline,
    VJEPABaseline
)

# Import HiMAC-JEPA
from src.models.himac_jepa import HiMACJEPA

print("✓ All models imported successfully")

## Configuration

In [ ]:
# Output directory
vis_dir = project_root / 'results/baselines/visualizations'
vis_dir.mkdir(parents=True, exist_ok=True)

# Device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Models to visualize
MODELS = ['camera_only', 'lidar_only', 'ijepa', 'vjepa', 'himac_jepa']

print(f"Visualization directory: {vis_dir}")
print(f"Device: {DEVICE}")

## 1. Trajectory Prediction Visualization

In [ ]:
def generate_dummy_trajectories(num_models=5, horizon_steps=30):
    """Generate dummy trajectory predictions for visualization."""

    # Ground truth trajectory (smooth curve)
    t = np.linspace(0, 3, horizon_steps)
    gt_x = t * 10 + np.sin(t * 2) * 2
    gt_y = t * 5 + np.cos(t * 3) * 1.5

    # Predicted trajectories (with varying accuracy)
    predictions = {}
    noise_levels = [0.15, 0.3, 0.5, 0.8, 1.2]  # Increasing noise for different models

    model_names = ['himac_jepa', 'vjepa', 'ijepa', 'camera_only', 'lidar_only']

    for i, (model_name, noise) in enumerate(zip(model_names[:num_models], noise_levels)):
        pred_x = gt_x + np.random.normal(0, noise, horizon_steps).cumsum() * 0.1
        pred_y = gt_y + np.random.normal(0, noise, horizon_steps).cumsum() * 0.1
        predictions[model_name] = (pred_x, pred_y)

    return (gt_x, gt_y), predictions


# Generate trajectories
gt_traj, pred_trajs = generate_dummy_trajectories()
gt_x, gt_y = gt_traj

# Plot
fig, ax = plt.subplots(figsize=(12, 10))

# Plot ground truth
ax.plot(gt_x, gt_y, 'k-', linewidth=4, label='Ground Truth', zorder=10)
ax.plot(gt_x[0], gt_y[0], 'go', markersize=15, label='Start', zorder=11)
ax.plot(gt_x[-1], gt_y[-1], 'r*', markersize=20, label='Goal', zorder=11)

# Plot predictions
colors = ['#8c564b', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
linestyles = ['-', '-', '--', '-.', ':']

for i, (model_name, (pred_x, pred_y)) in enumerate(pred_trajs.items()):
    ax.plot(pred_x, pred_y, linestyle=linestyles[i], color=colors[i], 
           linewidth=2.5, label=f'{model_name} prediction', alpha=0.8)

    # Show final position error
    final_error = np.sqrt((pred_x[-1] - gt_x[-1])**2 + (pred_y[-1] - gt_y[-1])**2)
    ax.plot(pred_x[-1], pred_y[-1], 'o', color=colors[i], markersize=10)

# Styling
ax.set_xlabel('X Position (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Y Position (m)', fontsize=14, fontweight='bold')
ax.set_title('Trajectory Predictions Comparison (3s Horizon)', fontsize=16, fontweight='bold')
ax.legend(fontsize=12, loc='upper left', frameon=True, shadow=True)
ax.grid(True, alpha=0.3, linestyle='--')
ax.axis('equal')

plt.tight_layout()
plt.savefig(vis_dir / 'trajectory_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {vis_dir / 'trajectory_predictions.png'}")

## 2. BEV Segmentation Visualization

In [ ]:
def generate_dummy_bev(size=(200, 200)):
    """Generate dummy BEV segmentation masks."""

    # Classes: 0=background, 1=road, 2=lane, 3=vehicle, 4=pedestrian
    H, W = size

    # Ground truth
    gt = np.zeros((H, W), dtype=np.uint8)
    gt[50:150, 80:120] = 1  # Road
    gt[70:130, 98:102] = 2  # Lane marking
    gt[60:80, 85:95] = 3    # Vehicle
    gt[120:140, 105:115] = 4 # Pedestrian

    # Predictions (with varying quality)
    predictions = {}

    # HiMAC-JEPA (best)
    pred = gt.copy()
    noise_mask = np.random.rand(H, W) > 0.97
    pred[noise_mask] = np.random.randint(0, 5, noise_mask.sum())
    predictions['himac_jepa'] = pred

    # V-JEPA (very good)
    pred = gt.copy()
    noise_mask = np.random.rand(H, W) > 0.95
    pred[noise_mask] = np.random.randint(0, 5, noise_mask.sum())
    predictions['vjepa'] = pred

    # I-JEPA (good)
    pred = gt.copy()
    noise_mask = np.random.rand(H, W) > 0.90
    pred[noise_mask] = np.random.randint(0, 5, noise_mask.sum())
    predictions['ijepa'] = pred

    # Camera-only (medium)
    pred = gt.copy()
    noise_mask = np.random.rand(H, W) > 0.80
    pred[noise_mask] = np.random.randint(0, 5, noise_mask.sum())
    predictions['camera_only'] = pred

    # LiDAR-only (poor semantics)
    pred = gt.copy()
    noise_mask = np.random.rand(H, W) > 0.60
    pred[noise_mask] = np.random.randint(0, 5, noise_mask.sum())
    predictions['lidar_only'] = pred

    return gt, predictions


# Generate BEV masks
gt_bev, pred_bevs = generate_dummy_bev()

# Color map for classes
cmap_colors = [
    [0.2, 0.2, 0.2],    # Background - dark gray
    [0.5, 0.5, 0.5],    # Road - gray
    [1.0, 1.0, 0.0],    # Lane - yellow
    [0.0, 0.0, 1.0],    # Vehicle - blue
    [1.0, 0.0, 0.0],    # Pedestrian - red
]
from matplotlib.colors import ListedColormap
cmap = ListedColormap(cmap_colors)

# Plot
fig, axes = plt.subplots(1, 6, figsize=(24, 4))

# Ground truth
axes[0].imshow(gt_bev, cmap=cmap, vmin=0, vmax=4)
axes[0].set_title('Ground Truth', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Predictions
for idx, (model_name, pred) in enumerate(pred_bevs.items()):
    axes[idx+1].imshow(pred, cmap=cmap, vmin=0, vmax=4)

    # Compute IoU (simplified)
    iou = (pred == gt_bev).mean()

    axes[idx+1].set_title(f'{model_name}\nIoU: {iou:.3f}', fontsize=14, fontweight='bold')
    axes[idx+1].axis('off')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=cmap_colors[0], label='Background'),
    Patch(facecolor=cmap_colors[1], label='Road'),
    Patch(facecolor=cmap_colors[2], label='Lane'),
    Patch(facecolor=cmap_colors[3], label='Vehicle'),
    Patch(facecolor=cmap_colors[4], label='Pedestrian'),
]
fig.legend(handles=legend_elements, loc='lower center', ncol=5, fontsize=12, frameon=True)

plt.suptitle('BEV Segmentation Predictions', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.savefig(vis_dir / 'bev_segmentation.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {vis_dir / 'bev_segmentation.png'}")

## 3. Multi-Agent Motion Prediction

In [ ]:
def generate_dummy_multi_agent_scene(num_agents=5):
    """Generate dummy multi-agent motion prediction scene."""
    
    # Ego vehicle
    ego_pos = np.array([0, 0])
    ego_heading = 0  # radians
    
    # Surrounding agents (position, heading, velocity)
    agents = []
    np.random.seed(42)
    
    for i in range(num_agents):
        pos = np.random.uniform(-20, 20, 2)
        heading = np.random.uniform(0, 2*np.pi)
        velocity = np.random.uniform(2, 10)
        agents.append({'pos': pos, 'heading': heading, 'velocity': velocity})
    
    # Generate future trajectories (3s, 0.5s steps = 6 steps)
    dt = 0.5
    steps = 6
    
    trajectories = {}
    for i, agent in enumerate(agents):
        traj = [agent['pos']]
        pos = agent['pos'].copy()
        heading = agent['heading']
        velocity = agent['velocity']
        
        for step in range(steps):
            # Simple constant velocity model + slight curve
            heading += np.random.normal(0, 0.05)
            pos += np.array([np.cos(heading), np.sin(heading)]) * velocity * dt
            traj.append(pos.copy())
        
        trajectories[i] = np.array(traj)
    
    return ego_pos, agents, trajectories


# Generate scene
ego_pos, agents, trajectories = generate_dummy_multi_agent_scene()

# Plot
fig, ax = plt.subplots(figsize=(12, 12))

# Plot ego vehicle
ego_rect = Rectangle(ego_pos - [2, 1], 4, 2, angle=0, 
                     facecolor='green', edgecolor='black', linewidth=2, label='Ego Vehicle')
ax.add_patch(ego_rect)
ax.plot(ego_pos[0], ego_pos[1], 'g*', markersize=20, zorder=10)

# Plot agents and trajectories
colors = plt.cm.tab10(np.linspace(0, 1, len(agents)))

for i, (agent, traj) in enumerate(zip(agents, trajectories.values())):
    pos = agent['pos']
    heading = agent['heading']
    
    # Agent bounding box
    rect = Rectangle(pos - [1.5, 0.75], 3, 1.5, 
                     angle=np.degrees(heading),
                     facecolor=colors[i], edgecolor='black', 
                     linewidth=1.5, alpha=0.7)
    ax.add_patch(rect)
    
    # Current position
    ax.plot(pos[0], pos[1], 'o', color=colors[i], markersize=10, zorder=5)
    
    # Predicted trajectory
    ax.plot(traj[:, 0], traj[:, 1], '--', color=colors[i], 
           linewidth=2, alpha=0.8, label=f'Agent {i+1}')
    ax.plot(traj[-1, 0], traj[-1, 1], 'x', color=colors[i], 
           markersize=12, markeredgewidth=3, zorder=5)

# Styling
ax.set_xlabel('X Position (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Y Position (m)', fontsize=14, fontweight='bold')
ax.set_title('Multi-Agent Motion Prediction (3s Horizon)', fontsize=16, fontweight='bold')
ax.legend(fontsize=11, loc='upper right', frameon=True, shadow=True)
ax.grid(True, alpha=0.3, linestyle='--')
ax.axis('equal')
ax.set_xlim([-30, 30])
ax.set_ylim([-30, 30])

# Add reference circle
circle = Circle((0, 0), 20, fill=False, edgecolor='gray', 
               linestyle=':', linewidth=2, alpha=0.5, label='20m range')
ax.add_patch(circle)

plt.tight_layout()
plt.savefig(vis_dir / 'multi_agent_motion.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {vis_dir / 'multi_agent_motion.png'}")

## 4. Latent Space Visualization (t-SNE)

In [ ]:
try:
    from sklearn.manifold import TSNE
    tsne_available = True
except ImportError:
    print("⚠️  scikit-learn not available, skipping t-SNE visualization")
    tsne_available = False

if tsne_available:
    # Generate dummy latent representations
    np.random.seed(42)

    num_samples = 200
    latent_dim = 256

    # Simulate latent representations from different models
    model_latents = {}

    for i, model_name in enumerate(['himac_jepa', 'vjepa', 'ijepa', 'camera_only', 'lidar_only']):
        # Each model has slightly different distribution
        center = np.random.randn(latent_dim) * 0.5
        latents = np.random.randn(num_samples, latent_dim) * 0.3 + center
        model_latents[model_name] = latents

    # Combine all latents
    all_latents = np.vstack(list(model_latents.values()))
    all_labels = np.concatenate([np.full(num_samples, i) 
                                 for i in range(len(model_latents))])

    # Apply t-SNE
    print("Running t-SNE (this may take a moment)...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    latents_2d = tsne.fit_transform(all_latents)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 10))

    colors = ['#8c564b', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    model_names = list(model_latents.keys())

    for i, (model_name, color) in enumerate(zip(model_names, colors)):
        mask = all_labels == i
        ax.scatter(latents_2d[mask, 0], latents_2d[mask, 1], 
                  c=color, label=model_name, s=50, alpha=0.6, edgecolors='black', linewidth=0.5)

    ax.set_xlabel('t-SNE Dimension 1', fontsize=14, fontweight='bold')
    ax.set_ylabel('t-SNE Dimension 2', fontsize=14, fontweight='bold')
    ax.set_title('Latent Space Visualization (t-SNE)', fontsize=16, fontweight='bold')
    ax.legend(fontsize=12, frameon=True, shadow=True, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--')

    plt.tight_layout()
    plt.savefig(vis_dir / 'latent_space_tsne.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"✓ Saved to {vis_dir / 'latent_space_tsne.png'}")

## 5. Summary Report

In [ ]:
print("\n" + "="*80)
print(" "*20 + "VISUALIZATION SUMMARY")
print("="*80 + "\n")

print("Generated visualizations:")
print("-"*80)
print(f"  1. Trajectory predictions     → {vis_dir / 'trajectory_predictions.png'}")
print(f"  2. BEV segmentation          → {vis_dir / 'bev_segmentation.png'}")
print(f"  3. Multi-agent motion        → {vis_dir / 'multi_agent_motion.png'}")
if tsne_available:
    print(f"  4. Latent space (t-SNE)      → {vis_dir / 'latent_space_tsne.png'}")

print("\n" + "="*80)
print("\nAll visualizations saved to:", vis_dir)
print("\nThese are dummy visualizations for demonstration.")
print("Replace with actual model predictions for real results.")
print("="*80)

## Next Steps

Visualization complete!

**To use with real data:**
1. Replace dummy data generation with actual dataset samples
2. Load trained models and extract predictions
3. Visualize real predictions vs ground truth

**All notebook suite complete:**
- ✅ `01_train_baselines.ipynb` - Training
- ✅ `02_evaluate_baselines.ipynb` - Evaluation
- ✅ `03_results_analysis.ipynb` - Analysis & plots
- ✅ `04_visualize_predictions.ipynb` - Qualitative visualization